In [ ]:
!pip install langchain openai
!pip install torch torch-cluster
!pip install python-arango
!pip install langchain-community
!pip install langchain-openai
!pip install --upgrade langchain langchain-community langchain-openai langgraph
!pip install langgraph
!pip install biomart

# !pip install fairseq==0.10.2
# !pip install torch_geometric
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
# !pip install rdkit==2024.03.1
# !pip install tensorboardX
# !pip install einops


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached fairseq-0.10.2.tar.gz (938 kB)
  Installing build dependencies ... done
  Getting

In [2]:
import os
import sys
import requests
import ast
from io import StringIO

import pandas as pd
from dotenv import load_dotenv
from arango import ArangoClient
from biomart import BiomartServer

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from DeepPurpose import utils
from DeepPurpose import DTI as models
from langchain_core.tools import tool

ModuleNotFoundError: No module named 'biomart'

In [145]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [146]:
db = ArangoClient(hosts="http://localhost:8529").db('NeuThera', username='root', password='openSesame')
arango_graph = ArangoGraph(db)

In [ ]:
# Your existing AQL query tool
@tool
def text_to_aql(query: str):
    """Execute a Natural Language Query in ArangoDB, and return the result as text."""
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
        llm=llm,
        graph=arango_graph,  # Assuming arango_graph is already initialized
        verbose=True,
        allow_dangerous_requests=True
    )
    
    result = chain.invoke(query)

    return str(result["result"])

@tool
def fetch_pdb_from_ensp(ensp_ids):
    """
    Fetch PDB IDs for given ENSP (Ensembl Protein) IDs using BioMart API.
    
    :param ensp_ids: A list of ENSP IDs.
    :return: JSON containing ENSP IDs and corresponding PDB IDs.
    """
    
    print("FETCHING PDB")

    # Ensure input is a list
    if not isinstance(ensp_ids, list):
        raise ValueError("ensp_ids must be a list of ENSP IDs")

    if not ensp_ids:  # Check for empty list
        return []

    # Construct BioMart query
    query_text = f"""<?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE Query>
    <Query virtualSchemaName="default" formatter="TSV" header="1" uniqueRows="1">
        <Dataset name="hsapiens_gene_ensembl" interface="default">
            <Filter name="ensembl_peptide_id" value="{','.join(ensp_ids)}"/>
            <Attribute name="ensembl_peptide_id"/>
            <Attribute name="pdb"/>
        </Dataset>
    </Query>"""

    url = "https://www.ensembl.org/biomart/martservice"

    try:
        response = requests.post(url, data={"query": query_text})
        if response.status_code != 200:
            raise Exception(f"Error {response.status_code}: {response.text}")

        # Convert response to JSON
        data = response.text.strip()
        df = pd.read_csv(StringIO(data), sep="\t")

        # Check if DataFrame is empty
        if df.empty:
            print("No PDB IDs found for given ENSP IDs.")
            return []

        # Convert DataFrame to JSON
        return df.head(5).to_dict(orient="records")

    except Exception as e:
        print(f"Failed to fetch PDB IDs: {e}")
        return []

def predict_binding_affinity(X_drug, X_target):
    """
    Predicts the binding affinity for given drug and target sequences.

    Parameters:
    X_drug (list): List containing the SMILES representation of the drug.
    X_target (list): List containing the amino acid sequence of the protein target.

    Returns:
    float: Predicted binding affinity (log(Kd) or log(Ki)).
    """

    
    model = models.model_pretrained(path_dir='DTI_model')

   
    X_pred = utils.data_process(X_drug, X_target, 
                                drug_encoding='CNN', 
                                target_encoding='CNN', 
                                split_method='no_split')

   
    predictions = model.predict(X_pred)

    return predictions[0]


In [165]:
tools = [ text_to_aql, fetch_pdb_from_ensp ]

def query_graph(query):
    
    query_template = f"""
    user input: {query}

    !IMPORTANT!
    ALWAYS FIND PROTEINS RELATED TO USER QUERY AND FIND PDB IDs FOR ANY ENSP IDs

    MAKE SURE YOU ALWAYS OUTPUT TO THE USER THE DIRECT RESULT FROM
    THE TOOLING. NO EXTRA TEXT REQUIRED.

    INVALID OUTPUT (EXAMPLE):
    The proteins related to the ID XXXX are identified by the following keys: ENSP00000xxxx, ENSP00000xxxx

    VALID OUTPUT (EXAMPLE):
    [XXXX, XXXX]
    """
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
    app = create_react_agent(llm, tools)    
    final_state = app.invoke({"messages": [{"role": "user", "content": query_template}]})
    return final_state["messages"][-1].content

In [174]:
output = query_graph("What proteins can you find me related to mitochondrially encoded cytochrome c oxidase III?")

try:
    output = ast.literal_eval(output) if isinstance(output, str) else output
except (SyntaxError, ValueError) as e:
    print(f"Error parsing query result: {e}")
    output = []



> Entering new ArangoGraphQAChain chain...
AQL Query (1):
WITH gene, protein, gene-protein
FOR gene_doc IN gene
  FILTER gene_doc.gene_name == "mitochondrially encoded cytochrome c oxidase III"
  FOR edge IN gene-protein
    FILTER edge._from == gene_doc._id
    FOR protein_doc IN protein
      FILTER protein_doc._id == edge._to
      RETURN protein_doc

AQL Query Execution Error: 
syntax error, unexpected - operator near '-protein
FOR gene_doc IN gene
  ...' at position 2:25

AQL Query (2):
WITH gene, protein, `gene-protein`
FOR gene_doc IN gene
  FILTER gene_doc.gene_name == "mitochondrially encoded cytochrome c oxidase III"
  FOR edge IN `gene-protein`
    FILTER edge._from == gene_doc._id
    FOR protein_doc IN protein
      FILTER protein_doc._id == edge._to
      RETURN protein_doc

AQL Result:
[{'_key': 'ENSP00000354982', '_id': 'protein/ENSP00000354982', '_rev': '_jSVQEHu---'}]

> Finished chain.
FETCHING PDB


In [175]:
output

[{'Protein stable ID': 'ENSP00000354982', 'PDB ID': '5Z62'}]